In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [2]:
from scanpy import read_h5ad
import os

DATA_DIR = "/home/julian/Uni/MasterThesis/data"

corona_ann = read_h5ad(os.path.join(DATA_DIR, "COVAC_POSTQC.h5ad"))

In [3]:
corona_ann.obs["timepoint"]

new_index
AAACCCACAAGCCCAC-59     t3
AAACCCATCATCCTGC-59     t3
AAACCCATCTTGTTAC-59     t3
AAACGAAAGACAACTA-59     t3
AAACGAACAAGTGGGT-59     t3
                      ... 
TTTGTTGTCGGCCTTT-42    d28
TTTGTTGTCGGTAGAG-42    d28
TTTGTTGTCGTGGACC-42    d28
TTTGTTGTCTATCGGA-42    d28
TTTGTTGTCTGTTGGA-42    d28
Name: timepoint, Length: 491473, dtype: category
Categories (8, object): ['M6', 'd1', 'd28', 'd56', 't1', 't2', 't3', 't4']